In [55]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly.plotly as py
import datetime
from __future__ import division

from moztelemetry import get_pings, get_pings_properties, get_one_ping_per_client

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [56]:
# We grab pings starting from 2 days ago until [TimeWindow] days ago.
TimeWindow = 1

# Additional filters.
Filters = {
    'app': 'Firefox',
    
    # We sample 0.5% of pings. For testing, it is beter to use a small number
    # here (like 0.001) to speed up processing time.
    'fraction': 1,
    
    # Optionally restrict pings to a single channel.
     'channel': 'nightly',
}

# In case you need them!
VendorIDs = {
    'Intel': '0x8086',
    'NVIDIA': '0x10de',
    'AMD': '0x1002',
    'ATI': '0x1002'
}

In [57]:
###############################
# This section gathers pings. #
###############################

def run_get_pings():
    def fmt_date(d):
        return d.strftime("%Y%m%d")
    t1 = fmt_date(datetime.datetime.now() - datetime.timedelta(TimeWindow + 0)) # go back 16 days
    t2 = fmt_date(datetime.datetime.now() - datetime.timedelta(-1)) # go back 2 days
    return get_pings(sc, build_id=(t1, t2), **Filters)

# Get pings for the parameters in the previous step.
raw_pings = run_get_pings()

In [58]:
######################################################################
# This section takes the raw ping list, then formats and filters it. #
######################################################################

# Map the pings into a more readable dictionary-like form. To see
# what these look like, execute "pings.take(1)".
unique_pings = get_pings_properties(raw_pings, [
  "clientId",
  "application/buildId",
  "environment/build/version",
  "environment/system/os/name",
  "environment/system/os/version",
  "environment/system/os/servicePackMajor",
  "environment/system/os/servicePackMinor",
  "environment/system/gfx/adapters",
  "environment/system/gfx/features",
  "payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID",
  "payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID"
])
unique_pings = get_one_ping_per_client(unique_pings)

In [59]:
# We add two extra steps. The first rewrites the ping to have some
# information more easily accessible (like the primary adapter),
# and the second step removes any pings that don't have adapter
# information.
def rewrite_ping(p):
    adapters = p.get('environment/system/gfx/adapters', None)
    if not adapters:
        return None
    adapter = adapters[0]
            
    p['adapter'] = adapter
    
    features = p.get('environment/system/gfx/features', None)
    if features == None:
        return None
            
    p['features'] = features
            
    # Convert the version to a tuple of integers.
    if 'driverVersion' in adapter:
        p['driverVersion'] = [int(n) for n in adapter['driverVersion'].split('.') if n.isdigit()]
    return p

def filter_ping(p):
    return p and 'adapter' in p and 'features' in p
pings = unique_pings.map(rewrite_ping).filter(filter_ping)
pings = pings.cache()
# Observe the format of a random ping. This may take some time since it has to
# execute the pipeline.
#pings.take(1)


In [60]:
# Count the total number of sessions in the dataset.
TotalSessions = pings.count()
print('Number of sessions: {0}'.format(TotalSessions))

Number of sessions: 8037


In [62]:
pings.map(lambda p : p['application/buildId']).countByValue()
#def fmt_date(d):
#    return d.strftime("%Y%m%d")
#fmt_date(datetime.datetime.now()  - datetime.timedelta(-1))

defaultdict(int, {u'20160710030217': 7989, u'20160711034039': 48})

In [63]:
import json
import urllib2

gpu_db = json.load(urllib2.urlopen('https://raw.githubusercontent.com/jrmuizel/gpu-db/master/intel.json'))
devices = {}
for gen in gpu_db['8086'].items():
    for chipset in gen[1].items():
        for dev in chipset[1]:
            #print dev, gen[0]
            devices[int(dev,16)] = gen[0]

In [64]:
# Sample filter #1 - how many people are using Intel devices
# with a driver less than 8.15.10.2622? (bug 1175366).
def sample_filter_1(p):
    if not 'vendorID' in p['adapter']:
        return False
    if not 'driver' in p['adapter']:
        return False

    if p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'] is None:
        return False
    #if devices[int(p['adapter']['deviceID'],16)] in ('gen6'):
    #    return True

    #if devices[int(p['adapter']['deviceID'],16)] in ('gen7'):

     #   return True
    return True

webgl_interesting_pings = pings.filter(sample_filter_1)
print(webgl_interesting_pings.count())
interesting_pings = webgl_interesting_pings.filter(lambda p: not p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'] is None)
print(interesting_pings.count())
interesting_pings.take(1)


224
222


[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) HD Graphics 4400',
   u'deviceID': u'0x041e',
   u'driver': u'igdumdim64 igd10iumd64 igd10iumd64 igd12umd64 igdumdim32 igd10iumd32 igd10iumd32 igd12umd32',
   u'driverDate': u'6-13-2016',
   u'driverVersion': u'20.19.15.4474',
   u'subsysID': u'78171462',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160710030217',
  'clientId': u'6e110751-742c-4fea-bf03-c01c298ebb5a',
  'driverVersion': [20, 19, 15, 4474],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) HD Graphics 4400',
    u'deviceID': u'0x041e',
    u'driver': u'igdumdim64 igd10iumd64 igd10iumd64 igd12umd64 igdumdim32 igd10iumd32 igd10iumd32 igd12umd32',
    u'driverDate': u'6-13-2016',
    u'driverVersion': u'20.19.15.4474',
    u'subsysID': u'78171462',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'd3d11',
   u'd2d': {u'status': u'ava

In [65]:
webgl_fail = webgl_interesting_pings.filter(lambda p: 'SUCCESS' not in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'])
webgl_fail.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()

defaultdict(int,
            {u'FEATURE_FAILURE_NO_DISPLAY': 4,
             u'FEATURE_FAILURE_WARP_FALLBACK': 5,
             u'FEATURE_FAILURE_WEBGL_DISABLED': 2})

In [66]:
nodisplay_fail = webgl_fail.filter(lambda p: 'FEATURE_FAILURE_NO_DISPLAY' in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'])
nodisplay_fail.take(4)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) 82945G Express Chipset Family',
   u'deviceID': u'0x2772',
   u'driver': u'igdumd32',
   u'driverDate': u'9-23-2009',
   u'driverVersion': u'8.15.10.1930',
   u'subsysID': u'00000000',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160710030217',
  'clientId': u'2877bb89-acca-4254-baca-72e9214e9804',
  'driverVersion': [8, 15, 10, 1930],
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) 82945G Express Chipset Family',
    u'deviceID': u'0x2772',
    u'driver': u'igdumd32',
    u'driverDate': u'9-23-2009',
    u'driverVersion': u'8.15.10.1930',
    u'subsysID': u'00000000',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'blacklisted', u'version': u'1.1'},
   u'd3d11': {u'status': u'failed'}},
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/s

In [67]:
accl_fail = interesting_pings.filter(lambda p: not '' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'])
print accl_fail.count()
accl_fail.take(5)

8


[{'adapter': {u'GPUActive': True,
   u'deviceID': u'0x0000',
   u'driver': u'vga framebuf vga256 vga64k',
   u'subsysID': u'00000000',
   u'vendorID': u'0x0000'},
  'application/buildId': u'20160710030217',
  'clientId': u'42982cd7-592c-4ded-a67e-75ceee371ad4',
  'environment/build/version': u'50.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'deviceID': u'0x0000',
    u'driver': u'vga framebuf vga256 vga64k',
    u'subsysID': u'00000000',
    u'vendorID': u'0x0000'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
   u'd3d11': {u'status': u'blacklisted'}},
  'environment/system/os/name': u'Windows_NT',
  'environment/system/os/servicePackMajor': 3,
  'environment/system/os/servicePackMinor': 0,
  'environment/system/os/version': u'5.1',
  'features': {u'compositor': u'basic',
   u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
   u'd3d11': {u'status': u'blacklisted'}},
  'payload

In [68]:
warp_success = accl_fail.filter(lambda p: 'SUCCESS' in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'])
print warp_success.count()
warp_success.map(lambda p: p['environment/system/os/version']).countByValue()

3


defaultdict(int, {u'10.0': 1, u'6.1': 1, u'6.2': 1})

In [69]:
warp_fail = accl_fail.filter(lambda p: 'SUCCESS' not in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'])
print warp_fail.count()
warp_fail.map(lambda p: p['environment/system/os/version']).countByValue()

5


defaultdict(int, {u'5.1': 4, u'5.2': 1})

In [71]:
print accl_fail.count()
accl_fail.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'].keys()[0],)).countByKey()

8


defaultdict(int,
            {u'FEATURE_FAILURE_AMD1': 1,
             u'FEATURE_FAILURE_DL_BLACKLIST_g1057': 2,
             u'FEATURE_FAILURE_TEST': 5})

In [72]:
accl7_fail = accl_fail.filter(lambda p: float(p['environment/system/os/version']) > 5.1)
accl7_fail.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'].keys()[0],)).countByKey()

defaultdict(int, {u'FEATURE_FAILURE_TEST': 4})

In [33]:
dl_fail = accl_fail.filter(lambda p: 'FEATURE_FAILURE_DL_BLACKLIST_g1057' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'])
dl_fail.map(lambda p: p['adapter']['driverVersion']).countByValue()

defaultdict(int,
            {u'6.13.10.3510': 1,
             u'6.14.10.3732': 1,
             u'6.14.10.3856': 1,
             u'6.14.10.4396': 2,
             u'6.14.10.4436': 2,
             u'6.14.10.4704': 1,
             u'6.14.10.4764': 1,
             u'6.14.10.4860': 1,
             u'6.14.10.4864': 3,
             u'6.14.10.4906': 1,
             u'6.14.10.4926': 4,
             u'6.14.10.4964': 1,
             u'6.14.10.5009': 1,
             u'6.14.10.5082': 1,
             u'6.14.10.5090': 1,
             u'6.14.10.5102': 1,
             u'6.14.10.5189': 1})

In [20]:
amd_fail = accl_fail.filter(lambda p: 'FEATURE_FAILURE_AMD1' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'])
print amd_fail.map(lambda p: p['adapter']['driverVersion']).countByValue()
print amd_fail.map(lambda p: p['adapter']['deviceID']).countByValue()

print amd_fail.map(lambda p: p['environment/system/os/version']).countByValue()

defaultdict(<type 'int'>, {u'8.475.0.0': 1, u'6.14.10.6497': 1, u'8.17.0.0': 1, u'8.501.0.0': 1, u'8.403.0.0': 1, u'8.479.0.0': 1, u'8.530.0.0': 1, u'8.204.0.0': 1, u'8.522.0.0': 1})
defaultdict(<type 'int'>, {u'0x718a': 1, u'0x791e': 1, u'0x7942': 2, u'0x95c4': 1, u'0x71c5': 1, u'0x4437': 1, u'0x5b63': 1, u'0x5954': 1})
defaultdict(<type 'int'>, {u'5.1': 7, u'6.1': 2})


In [16]:
nv_fail = accl_fail.filter(lambda p: 'FEATURE_FAILURE_NV_W7' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'] or 'FEATURE_FAILURE_NV_XP' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'])

print nv_fail.map(lambda p: p['adapter']['deviceID']).countByValue()

print nv_fail.map(lambda p: p['environment/system/os/version']).countByValue()
nv_fail.map(lambda p: p['adapter']['driverVersion']).countByValue()

defaultdict(<type 'int'>, {u'0x0658': 1, u'0x02e2': 1, u'0x0a75': 2, u'0x06ec': 1, u'0x0322': 2, u'0x0402': 1, u'0x0427': 1, u'0x01d3': 1, u'0x03d0': 1, u'0x040c': 1, u'0x0622': 1, u'0x0429': 1, u'0x03d6': 3})
defaultdict(<type 'int'>, {u'5.1': 10, u'6.1': 7})


defaultdict(int,
            {u'5.6.7.3': 1,
             u'6.1.7600.16385': 1,
             u'6.14.10.9382': 1,
             u'6.14.11.6218': 1,
             u'6.14.11.6757': 1,
             u'6.14.11.6921': 1,
             u'6.14.11.7474': 1,
             u'6.14.11.7924': 1,
             u'6.14.11.8224': 3,
             u'8.15.11.8593': 1,
             u'8.15.11.8624': 1,
             u'8.15.11.8644': 1,
             u'8.16.11.8886': 1,
             u'8.16.11.8950': 1,
             u'8.16.11.9181': 1})

In [22]:
test_fail = accl_fail.filter(lambda p: 'FEATURE_FAILURE_TEST' in p['payload/keyedHistograms/CANVAS_WEBGL_ACCL_FAILURE_ID'])
print test_fail.map(lambda p: p['adapter']['vendorID']).countByValue()

defaultdict(<type 'int'>, {u'0x15ad': 3, u'0x00ba': 2, u'0x0000': 9, u'0x1ab8': 1, u'0x80ee': 1, u'0x1106': 1})


In [14]:
def webgl_fail(p):
    if not 'SUCCESS' in p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID']:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)

extracted = fail_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],))
print fail_pings.count()

extracted.countByKey()


198


defaultdict(int,
            {u'FEATURE_FAILURE_EGL_LIB_INIT': 188,
             u'FEATURE_FAILURE_EGL_OFFSCREEN': 1,
             u'FEATURE_FAILURE_PERF_WARP': 6,
             u'FEATURE_FAILURE_WEBGL_DISABLED': 3})

In [15]:
fail_pings.take(5)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) G33/G31 Express Chipset Family',
   u'deviceID': u'0x29c2',
   u'driver': u'igxprd32',
   u'driverDate': u'11-3-2008',
   u'driverVersion': u'6.14.10.5009',
   u'subsysID': u'00000000',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160625004039',
  'clientId': u'cc3da994-b2ee-4431-a5b0-73f301329c89',
  'driverVersion': [6, 14, 10, 5009],
  'environment/build/version': u'49.0a2',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) G33/G31 Express Chipset Family',
    u'deviceID': u'0x29c2',
    u'driver': u'igxprd32',
    u'driverDate': u'11-3-2008',
    u'driverVersion': u'6.14.10.5009',
    u'subsysID': u'00000000',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'unavailable', u'version': u'1.1'},
   u'd3d11': {u'status': u'blacklisted'}},
  'environment/system/os/name': u'Windows_NT',
  'environment/syst

In [16]:
win7_pings = fail_pings.filter(lambda p: float(p['environment/system/os/version']) > 5.1)
win7_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()


defaultdict(int,
            {u'FEATURE_FAILURE_EGL_LIB_INIT': 34,
             u'FEATURE_FAILURE_EGL_OFFSCREEN': 1,
             u'FEATURE_FAILURE_PERF_WARP': 6,
             u'FEATURE_FAILURE_WEBGL_DISABLED': 3})

In [17]:
def expected_dis(p):
    k = p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()
    if k == ['FEATURE_FAILURE_PERF_WARP']:
        return False
    if k == ['FEATURE_FAILURE_WEBGL_DISABLED']:
        return False
    return True
interesting_win7 = win7_pings.filter(expected_dis)
interesting_win7.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()


defaultdict(int,
            {u'FEATURE_FAILURE_EGL_LIB_INIT': 34,
             u'FEATURE_FAILURE_EGL_OFFSCREEN': 1})

In [24]:
interesting_win7.take(4)

[{'adapter': {u'GPUActive': True,
   u'description': u'Radeon X300/X550/X1050 Series (Microsoft Corporation - WDDM) ',
   u'deviceID': u'0x5b60',
   u'driver': u'atiumdag atiumdva atiumd64 atiumd6a atitmm64',
   u'driverDate': u'4-25-2009',
   u'driverVersion': u'8.56.1.16',
   u'subsysID': u'0f021002',
   u'vendorID': u'0x1002'},
  'application/buildId': u'20160625004039',
  'clientId': u'fcfdd6b0-d1a8-4c80-8b7e-88bec1418dd2',
  'driverVersion': [8, 56, 1, 16],
  'environment/build/version': u'49.0a2',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Radeon X300/X550/X1050 Series (Microsoft Corporation - WDDM) ',
    u'deviceID': u'0x5b60',
    u'driver': u'atiumdag atiumdva atiumd64 atiumd6a atitmm64',
    u'driverDate': u'4-25-2009',
    u'driverVersion': u'8.56.1.16',
    u'subsysID': u'0f021002',
    u'vendorID': u'0x1002'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'failed', u'version': u'1.1'},
   u'd3

In [25]:
win7_intel_pings = interesting_win7.filter(lambda p: p['adapter']['vendorID'] == '0x8086')
def do_gen(p):
    p['gen'] = devices[int(p['adapter']['deviceID'],16)]
    return p
    
win7_intel_pings = win7_intel_pings.map(do_gen)
win7_intel_pings.map(lambda p: (p['gen'],)).countByKey()

defaultdict(int, {u'gen3': 21, u'gen4': 1, u'gen6': 2, u'gen7': 6})

In [27]:
recent_intel = win7_intel_pings.filter(lambda p: p['gen'] in ('gen6','gen7') and not 'vga' in p['adapter']['driver'])

recent_intel.take(4)

[{'adapter': {u'GPUActive': True,
   u'description': u'Intel(R) HD Graphics Family',
   u'deviceID': u'0x0102',
   u'driver': u'igdumdx32 igd10umd32 igd10umd32',
   u'driverDate': u'12-29-2010',
   u'driverVersion': u'8.15.10.2274',
   u'subsysID': u'00000000',
   u'vendorID': u'0x8086'},
  'application/buildId': u'20160626004021',
  'clientId': u'b9a5047c-c2fa-46bd-8004-7712a1be0083',
  'driverVersion': [8, 15, 10, 2274],
  'environment/build/version': u'49.0a2',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Intel(R) HD Graphics Family',
    u'deviceID': u'0x0102',
    u'driver': u'igdumdx32 igd10umd32 igd10umd32',
    u'driverDate': u'12-29-2010',
    u'driverVersion': u'8.15.10.2274',
    u'subsysID': u'00000000',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'status': u'blacklisted', u'version': u'1.1'},
   u'd3d11': {u'status': u'failed'}},
  'environment/system/os/name': u'Windows_NT',


In [19]:
xp_pings = fail_pings.filter(lambda p: float(p['environment/system/os/version']) == 5.1)
xp_pings.map(lambda p: (p['payload/keyedHistograms/CANVAS_WEBGL_FAILURE_ID'].keys()[0],)).countByKey()

defaultdict(int, {u'FEATURE_FAILURE_EGL_LIB_INIT': 154})

In [20]:
xp_pings.map(lambda p: (p['adapter']['vendorID'],)).countByKey()

defaultdict(int,
            {u'0x0000': 19,
             u'0x1002': 21,
             u'0x1039': 4,
             u'0x10de': 10,
             u'0x1106': 1,
             u'0x5333': 1,
             u'0x8086': 98})

In [53]:
not_vga_pings.map(lambda p: (p['adapter']['vendorID'],)).countByKey()

defaultdict(int, {u'0x1002': 5, u'0x10de': 6, u'0x8086': 16})

In [21]:
xp_intel_pings = xp_pings.filter(lambda p: p['adapter']['vendorID'] == '0x8086')
def do_gen(p):
    p['gen'] = devices[int(p['adapter']['deviceID'],16)]
    return p
    
xp_intel_pings = xp_intel_pings.map(do_gen)
xp_intel_pings.map(lambda p: (p['gen'],)).countByKey()

defaultdict(int, {u'gen2': 11, u'gen3': 67, u'gen4': 19, u'powervr': 1})

In [22]:
intel_pings = not_vga_pings.filter(lambda p: p['adapter']['vendorID'] == '0x8086')
nv_pings = not_vga_pings.filter(lambda p: p['adapter']['vendorID'] == '0x10de')
def do_gen(p):
    p['gen'] = devices[int(p['adapter']['deviceID'],16)]
    return p
    
intel_pings = intel_pings.map(do_gen)
intel_pings.map(lambda p: (p['gen'],)).countByKey()

defaultdict(int,
            {u'gen2': 11,
             u'gen3': 88,
             u'gen4': 21,
             u'gen6': 2,
             u'gen7': 2,
             u'powervr': 1})

In [42]:
nv_pings.take(4)

[{'adapter': {u'GPUActive': True,
   u'RAM': 4095,
   u'description': u'NVIDIA GeForce GTX 970',
   u'deviceID': u'0x13c2',
   u'driver': u'nvd3dumx,nvwgf2umx,nvwgf2umx,nvwgf2umx nvd3dum,nvwgf2um,nvwgf2um,nvwgf2um',
   u'driverDate': u'4-27-2016',
   u'driverVersion': u'10.18.13.6510',
   u'subsysID': u'39753842',
   u'vendorID': u'0x10de'},
  'clientId': u'5a171441-eeba-4e82-a356-50120314b974',
  'driverVersion': [10, 18, 13, 6510],
  'environment/build/version': u'49.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'RAM': 4095,
    u'description': u'NVIDIA GeForce GTX 970',
    u'deviceID': u'0x13c2',
    u'driver': u'nvd3dumx,nvwgf2umx,nvwgf2umx,nvwgf2umx nvd3dum,nvwgf2um,nvwgf2um,nvwgf2um',
    u'driverDate': u'4-27-2016',
    u'driverVersion': u'10.18.13.6510',
    u'subsysID': u'39753842',
    u'vendorID': u'0x10de'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'failureId': u'OK', u'status': u'failed', u'version': u'1.1'},
   

In [57]:
intel_pings.filter(lambda p: p['gen'] == 'gen4').take(5)

[{'adapter': {u'GPUActive': True,
   u'description': u'Mobile Intel(R) 4 Series Express Chipset Family',
   u'deviceID': u'0x2a42',
   u'driver': u'igxprd32',
   u'driverDate': u'8-25-2008',
   u'driverVersion': u'6.14.10.4980',
   u'subsysID': u'024d1028',
   u'vendorID': u'0x8086'},
  'clientId': u'4b055f9c-aed1-4f50-88c8-cff8fc7d72e1',
  'driverVersion': [6, 14, 10, 4980],
  'environment/build/version': u'49.0a1',
  'environment/system/gfx/adapters': [{u'GPUActive': True,
    u'description': u'Mobile Intel(R) 4 Series Express Chipset Family',
    u'deviceID': u'0x2a42',
    u'driver': u'igxprd32',
    u'driverDate': u'8-25-2008',
    u'driverVersion': u'6.14.10.4980',
    u'subsysID': u'024d1028',
    u'vendorID': u'0x8086'}],
  'environment/system/gfx/features': {u'compositor': u'basic',
   u'd2d': {u'failureId': u'FEATURE_FAILURE_DL_BLACKLIST_g984',
    u'status': u'unavailable',
    u'version': u'1.1'},
   u'd3d11': {u'failureId': u'FEATURE_FAILURE_BUG_1153381',
    u'status': u'

In [ ]:
def webgl_fail(p):
    if p['payload/histograms/CANVAS_WEBGL_FAILURE_ID']['SUCCESS'] == 0:
        return True

     #   return True
    return False
fail_pings = interesting_pings.filter(webgl_fail)

def vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return True


     #   return True
    return False
vga_pings = fail_pings.filter(vga_fail)

def not_vga_fail(p):
    if 'vga' in p['adapter']['driver']:
        return False
     #   return True
    return True
not_vga_pings = fail_pings.filter(not_vga_fail)


amd1_pings = fail_pings.filter(lambda p: p['features']['webgl']['failureId'] == 'FEATURE_FAILURE_AMD1')



#TDRResults = sample_result_1.map(lambda p: p[DeviceResetReasonKey]).reduce(lambda x, y: x + y)
#help(pd.Series(TDRResults.values, DeviceResetReason))
#print(pd.Series(TDRResults.values[1:8],DeviceResetReason))
#pd.Series(TDRResults.values[1:8],DeviceResetReason).plot.bar()
#reduce(lambda x, y: sample_result_1.take(2))
#sample_result_1.take(1)[0]['payload/histograms/DEVICE_RESET_REASON'])
#fail_pings.take(1)
def match_count(a, b):
    print('{0} out of {1} sessions matched. ({2:.2f}%)'.format(
        a.count(),
        b.count(),
       ((a.count() / b.count()) * 100)))



def map_x_to_count(data, sourceKey):
    def extract(p):
        return (p['adapter'][sourceKey],)
    return data.map(extract).countByKey()    
match_count(interesting_pings, pings)    
match_count(fail_pings, interesting_pings)
match_count(vga_pings, fail_pings)
match_count(not_vga_pings, fail_pings)
match_count(amd1_pings, fail_pings)
#map_x_to_count(not_vga_pings, 'driverVersion')
#not_vga_pings.take(5)

#fail_pings.map(lambda p: (p['features']['webgl']['failureId'],)).countByKey()
#amd1_pings.take(23)
amd1_pings.map(lambda p: (p['adapter']['driverVersion'],)).countByKey()